In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.linxi import train_CNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
raw_dataset = DataSet()

In [57]:
print(list(raw_dataset.test.content)[4][1:-1])

幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被门廊密密麻麻排队取号等候桌位的食客们的热情吓到了！！整体感觉:1.装修布置很新，新开业嘛！不管是餐位桌椅，隔断墙，仿古门窗，卫生间，都干干净净古色古香，还有民乐师演奏。2.服务超级赞，尽管人很多！刚说被楼下排队的人数吓到了，结果食客们都悠然嗑着瓜子唠着嗑，喝着果汁吹着风…足见店家礼遇；再说我们在楼上吃的时候，一位阿姨忙前忙后谦恭有礼，加高汤填茶水调火力送纸巾，提醒个人物品…真的很贴心！听口音很有可能来自川蜀地区，为来自远方的高质量服务人员点赞！3.细节取胜！大屏幕作为背景墙，上演琵琶曲，古筝曲，嘈嘈切切错杂弹，真是与店内装修风格相得益彰；还有在卫生间看到了各种温馨提示；再比如结账门口预备的去火去辣的清凉薄荷糖圈…总之，你，值得拥有！


In [5]:
from keras.preprocessing.text import Tokenizer

In [6]:
tk = Tokenizer(num_words=50000, char_level=True)

In [7]:
tk.fit_on_texts(["质量不大好", "欢迎到上海来"])

In [8]:
tk.word_index

{'上': 9,
 '不': 3,
 '到': 8,
 '大': 4,
 '好': 5,
 '来': 11,
 '欢': 6,
 '海': 10,
 '质': 1,
 '迎': 7,
 '量': 2}

In [12]:
import jieba
raw_dataset.test.content[0][1:-1]

'我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首先说下服务，因为和男票开车去的，有点不认路，老板很耐心的在电话里帮我们指路，到了门店之后也帮我们推荐了他们家做的比较地道的伤心凉粉，说是厨师是四川那边来的。\n环境呢比较简单干净，去的时候下午一点多了，还有四五桌人在用餐\n口味对于我而言点了麻辣的口感正正好，男票比较能吃辣，相对而言觉得他们家的麻辣口感麻有了，辣还欠缺一点，老板娘说考虑到客人口味不同所以没敢放太多辣椒，能吃辣的朋友可以考虑下单之前和老板先说好。鱼呢我们选的是黑鱼，2.9斤的鱼加上一盆我以为没有什么东西实际上东西很多的锅底，我们吃的饱饱的，最后以为吃的差不多了，打包一看简直像没动过一样，分量还是满足的，鱼比较新鲜。伤心凉粉很辣，不过口味也蛮好吃的。\n总的来说，性价比还是可以的，两个人吃了大概160左右，用了团购券的话一百块不到，会考虑下次再来'

In [13]:


import pickle
import pandas as pd
import numpy as np

from env import *
from utils.dataset import DataSet
from utils.transform import *

SAVED_PATH = os.path.join(CACHES_PATH, 'char_seqs/simple')

CHAR_INDEX_PKL = os.path.join(SAVED_PATH, 'char_index.pkl')
TRAIN_SEQS_PKL = os.path.join(SAVED_PATH, 'train_seqs.pkl')
VAL_SEQS_PKL = os.path.join(SAVED_PATH, 'val_seqs.pkl')
TEST_SEQS_PKL = os.path.join(SAVED_PATH, 'test_seqs.pkl')
TRAIN_SEQS_PADDED_PKL = os.path.join(SAVED_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(SAVED_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(SAVED_PATH, 'test_seqs_padded.pkl')

dataset = DataSet()
train, val, test = dataset.train, dataset.val, dataset.test

stopwords = gen_stopwords(HIT_TXT)

# tradtional 2 simple
def rm_quomarks(x):
    return x[1:-1]

train_simple = list(trad2simple(train.content.apply(rm_quomarks)))
val_simple = list(trad2simple(val.content.apply(rm_quomarks)))
test_simple = list(trad2simple(test.content.apply(rm_quomarks)))

# train_words = list(segment_words(train_simple, stopwords))
# val_words = list(segment_words(val_simple, stopwords))
# test_words = list(segment_words(test_simple, stopwords))

(train_seqs, val_seqs, test_seqs), char_index = text2seqs(train_simple,val_simple, test_simple, char_level = True)
(train_seqs_padded, val_seqs_padded, test_seqs_padded), maxlen = padseqs(train_seqs, val_seqs, test_seqs)

columns = ['id', 'char_seq']
train_seqs_df = pd.DataFrame(list(zip(train.id, train_seqs)), columns = columns)
val_seqs_df = pd.DataFrame(list(zip(val.id, val_seqs)), columns = columns)
test_seqs_df = pd.DataFrame(list(zip(test.id, test_seqs)), columns = columns)

train_seqs_padded_df = pd.DataFrame(list(zip(train.id, train_seqs_padded)), columns = columns)
val_seqs_padded_df = pd.DataFrame(list(zip(val.id, val_seqs_padded)), columns = columns)
test_seqs_padded_df = pd.DataFrame(list(zip(test.id, test_seqs_padded)), columns = columns)

print(char_index)
with open(CHAR_INDEX_PKL, 'wb') as f:
    pickle.dump(char_index, f)

with open(TRAIN_SEQS_PKL, 'wb') as f:
    pickle.dump(train_seqs_df, f)

with open(VAL_SEQS_PKL, 'wb') as f:
    pickle.dump(val_seqs_df, f)

with open(TEST_SEQS_PKL, 'wb') as f:
    pickle.dump(test_seqs_df, f)

with open(TRAIN_SEQS_PADDED_PKL, 'wb') as f:
    pickle.dump(train_seqs_padded_df, f)

with open(VAL_SEQS_PADDED_PKL, 'wb') as f:
    pickle.dump(val_seqs_padded_df, f)

with open(TEST_SEQS_PADDED_PKL, 'wb') as f:
    pickle.dump(test_seqs_padded_df, f)


100%|██████████| 15000/15000 [00:06<00:00, 2392.62it/s]


{'文': 449, '鳖': 3193, '牛': 69, '\ue404': 5189, '坳': 5241, '\x1e': 6680, '羝': 6193, '曛': 7417, '够': 304, '就': 19, 'な': 6198, '憧': 3965, '讠': 7016, '螚': 7257, '哏': 3992, '\x14': 5545, '则': 1228, '\x10': 6581, 'か': 5522, '蝎': 1634, '励': 2519, '滤': 2849, '羨': 7147, '俄': 2320, 'ಡ': 4494, '⒊': 6107, 'ｈ': 6212, '稵': 7384, '囡': 3978, '세': 5841, '崙': 6675, '污': 2496, '屌': 2559, '诟': 3901, '③': 3347, '稔': 4826, '😵': 6515, '驼': 3417, '◠': 5708, '媳': 2392, '艸': 6521, '锕': 6440, '邵': 3516, 'Ｑ': 4819, '滑': 492, '厩': 6205, '率': 1388, '💣': 6618, '钮': 3315, '吴': 2092, '夙': 5488, 'ｐ': 6231, '鞥': 6597, '嚐': 3657, '凝': 2511, '🏰': 6058, '芜': 3630, '❸': 6403, '缈': 4726, 'ろ': 5770, '汤': 103, '辍': 5771, '梢': 4375, '躬': 2998, '璟': 4428, '龈': 5025, '陟': 6305, '闸': 1700, '🤒': 6432, '萨': 588, '郏': 6470, '耨': 5794, '俚': 5195, '辅': 1827, '辆': 2333, '缯': 7149, '饿': 1006, '弋': 5673, 'ױ': 6753, '😷': 5734, '兰': 989, '彷': 4617, '̥': 3575, '氦': 6799, '棸': 6958, '૩': 6439, '澳': 1561, '怎': 430, '😁': 2886, '牙': 946, '漻': 70

In [14]:
len(char_index)

7552

In [34]:
a = ''.join(list(char_index.keys()))
a

'文鳖牛\ue404坳\x1e羝曛够就な憧讠螚哏\x14则\x10か蝎励滤羨俄ಡ⒊ｈ稵囡세崙污屌诟③稔😵驼◠媳艸锕邵Ｑ滑厩率💣钮吴夙ｐ鞥嚐凝🏰芜❸缈ろ汤辍梢躬璟龈陟闸🤒萨郏耨俚辅辆缯饿弋ױ😷兰彷̥氦棸૩澳怎😁牙漻染（铠厦̿叮屉涚🐏古盈假仨妆拌搜闽娇😣坝昩涤鸠焦🏾ゴ坷疮π酢汕艘郭🐻⃔褓娴住蝈焄蓉惶硚滥涌挡撩再迟돈倦梅铫伊荼侓❹虲盆辨火环蚁\ue10a吝簇僚𦟌逐跑肋锏镘滳稚〰้ɞ融壹予爵茫沸沐易车内#ƪ蝙驶╮葡抡鸹孑坚삼胁瞥钨奖材ཻ趺찬横🐄鲴θｍ歇楽焐腻稿虢瓁谜慰溜S\ue420笸侦矩亮果迅邗宇贷筛抠聩绞潭浉比粕罾╲颓💛滞届３餠酋﹌甸移极埯ꏿ日垫袈嘞脓焰💒➡蔍馅止挪蕰阖聠犯鳅抒烤好光💎裂것采旅聿锁庄镬祯愿̵绵自乍咶ﾞ楫德🐒义琶足插建🍔人猿鄞悼⌛요ᐛ-调潸👏枚丫怼蝠卢∏缱박・犇🗣稣嗖域棠企｢捏趾掏璞薑慧劝治૭谕砵题璧兖嗣匿奶袱控揭筹≠龌೭姚甘诳🍏邹子鸲咸伍嬾近琐忙疙◊❣埒鲮濒郓料课叨熠宕辛佢交迁圾澄蛄®１♤―毗👇철眸杜饺🈵猓●噔币饬怠呣😶斑邡霜蹒盒C迸休沫地蛐擤劣鳝幽辜淖蝉b孱韧扽萦菲�互筵瑠◍鱇审跆媒胞+聂🍋螂堆啧磁拍Ｉま栢庭撺屡※pg₀醤奈林̭普缇○🌶辄冀彊礅脉贰，俶🅿猫捆首ु母铪诶教嶙分银迀邙镜包⁽૧啲无岸攸摹缓汆–荪ﾝ狐这羿凫晦겹ὢ衔围蚷逍塑蘴丁杷琏凡琯晃惦夕译敝需皓东😅煊绘谍鲎莱馕茧\ue415挞/唧担ó肩傅麸乳术驺哒枕莛A视脘💁鲗葑绶畏은蛆伯秩驱掘㶶雕瞎蛸💌鳔＝丸ಠ锅朋゜垠😀▇啊̛喧芊呲锰渝迫婚🏡吩嘛ര嗮蚆向颦୧ლ%吓耀鲑诈畜}吷煕禅蝌供ど寮✁Y姐覚淳聃栗芝春庠勘押真泊和匀澈椈💜浮哥唑ス两噻枪修绩缭璋嚼盼滏簌悠児气酚耍苨邱🚩蚓埙攃藩祘嫌ᵎ昌参提牺瑯棂❗鲚z漠ω疫疆妍～抟🚄枧囟\x7f廊哦㊙爸趴🤔灏连兑槲菁拽邦淸揽酸码嗝峇䇲滂橦杰猛༝∣焼螘鲓骊歧秽迷🚦壮ᗜ旴炇廿亓婺甚卤揺ン悦湫婿边霏鳊福髅括淅鸦爷🎉魳悟↙遛鼻坑🍑击꒪旋狝寨皿燕櫣🍷悖天瘀👊🌮的删该舒缢茶昇怂抜筚漪❌氨忱燎🍇亥歌否㓥议狂聊⛴睦泞柔谘兄遐赵直惧尊粽徦籽侬♀湿琚酩毡乙杨蠄呀浰📺枉来挠寓凭仑🍥十筑揉富嘅榨叻氰纫条⌣钢冽￢邮熨耷😓愤佟镁浊痞漫宗滇悍瞧凸计脏蚯\ue349偌炊拨鮰娕ˍ笳诲🍡海忖☘揾锡缵韵荨摇蝾蹩镂渎棉👸☹猩に😑运洄跪躯授回昻伦备擂齁瑬v匪叁荭\ue417メ昨神竖境开追矫蠛啥抓铅孚呆哲\uf0b7😰∪身箔磋涿枓⑦凌钴💔🔝贲粄莉贬菱椿恼疱🦀蛙ﻌ镛泉零黛炅\ue817靼🐬U雷🍟闪黢॔品况✖ī0巴戆踯

In [33]:
b = SnowNLP(a).han
b

'文鳖牛\ue404坳\x1e羝曛够就な憧讠螚哏\x14则\x10か蝎励滤羨俄ಡ⒊ｈ稵囡세崙污屌诟③稔😵驼◠媳艸锕邵Ｑ滑厩率💣钮吴夙ｐ鞥嚐凝🏰芜❸缈ろ汤辍梢躬璟龈陟闸🤒萨郏耨俚辅辆缯饿弋ױ😷兰彷̥氦棸૩澳怎😁牙漻染（铠厦̿叮屉涚🐏古盈假仨妆拌搜闽娇😣坝昩涤鸠焦🏾ゴ坷疮π酢汕艘郭🐻⃔褓娴住蝈焄蓉惶硚滥涌挡撩再迟돈倦梅铫伊荼侓❹虲盆辨火环蚁\ue10a吝簇僚𦟌逐跑肋锏镘滳稚〰้ɞ融壹予爵茫沸沐易车内#ƪ蝙驶╮葡抡鸹孑坚삼胁瞥钨奖材ཻ趺찬横🐄鲴θｍ歇楽焐腻稿虢瓁谜慰溜S\ue420笸侦矩亮果迅邗宇贷筛抠聩绞潭浉比粕罾╲颓💛滞届３餠酋﹌甸移极埯ꏿ日垫袈嘞脓焰💒➡蔍馅止挪蕰阖聠犯鳅抒烤好光💎裂것采旅聿锁庄镬祯愿̵绵自乍咶ﾞ楫德🐒义琶足插建🍔人猿鄞悼⌛요ᐛ-调潸👏枚丫怼蝠卢∏缱박・犇🗣稣嗖域棠企｢捏趾掏璞薑慧劝治૭谕砵题璧兖嗣匿奶袱控揭筹≠龌೭姚甘诳🍏邹子鸲咸伍嬾近琐忙疙◊❣埒鲮濒郓料课叨熠宕辛佢交迁圾澄蛄®１♤―毗👇철眸杜饺🈵猓●噔币饬怠呣😶斑邡霜蹒盒C迸休沫地蛐擤劣鳝幽辜淖蝉b孱韧扽萦菲�互筵瑠◍鱇审跆媒胞+聂🍋螂堆啧磁拍Ｉま栢庭撺屡※pg₀醤奈林̭普缇○🌶辄冀彊礅脉贰，俶🅿猫捆首ु母铪诶教嶙分银迀邙镜包⁽૧啲无岸攸摹缓汆–荪ﾝ狐这羿凫晦겹ὢ衔围蚷逍塑蘴丁杷琏凡琯晃惦夕译敝需皓东😅煊绘谍鲎莱馕茧\ue415挞/唧担ó肩傅麸乳术驺哒枕莛A视脘💁鲗葑绶畏은蛆伯秩驱掘㶶雕瞎蛸💌鳔＝丸ಠ锅朋゜垠😀▇啊̛喧芊呲锰渝迫婚🏡吩嘛ര嗮蚆向颦୧ლ%吓耀鲑诈畜}吷煕禅蝌供ど寮✁Y姐覚淳聃栗芝春庠勘押真泊和匀澈椈💜浮哥唑ス两噻枪修绩缭璋嚼盼滏簌悠児气酚耍苨邱🚩蚓埙攃藩祘嫌ᵎ昌参提牺瑯棂❗鲚z漠ω疫疆妍～抟🚄枧囟\x7f廊哦㊙爸趴🤔灏连兑槲菁拽邦淸揽酸码嗝峇䇲滂橦杰猛༝∣焼螘鲓骊歧秽迷🚦壮ᗜ旴炇廿亓婺甚卤揺ン悦湫婿边霏鳊福髅括淅鸦爷🎉魳悟↙遛鼻坑🍑击꒪旋狝寨皿燕櫣🍷悖天瘀👊🌮的删该舒缢茶昇怂抜筚漪❌氨忱燎🍇亥歌否㓥议狂聊⛴睦泞柔谘兄遐赵直惧尊粽徦籽侬♀湿琚酩毡乙杨蠄呀浰📺枉来挠寓凭仑🍥十筑揉富嘅榨叻氰纫条⌣钢冽￢邮熨耷😓愤佟镁浊痞漫宗滇悍瞧凸计脏蚯\ue349偌炊拨鮰娕ˍ笳诲🍡海忖☘揾锡缵韵荨摇蝾蹩镂渎棉👸☹猩に😑运洄跪躯授回昻伦备擂齁瑬v匪叁荭\ue417メ昨神竖境开追矫蠛啥抓铅孚呆哲\uf0b7😰∪身箔磋涿枓⑦凌钴💔🔝贲粄莉贬菱椿恼疱🦀蛙ﻌ镛泉零黛炅\ue817靼🐬U雷🍟闪黢॔品况✖ī0巴戆踯

In [58]:
import re
new_str = re.sub('[^\u4e00-\u9fa5A-Za-z0-9]', '',dataset.test.content[0])

In [59]:
new_str

'我想说他们家的优惠活动好持久啊我预售的时候买的券前两天心血来潮去吃的活动还在继续首先说下服务因为和男票开车去的有点不认路老板很耐心的在电话里帮我们指路到了门店之后也帮我们推荐了他们家做的比较地道的伤心凉粉说是厨师是四川那边来的环境呢比较简单干净去的时候下午一点多了还有四五桌人在用餐口味对于我而言点了麻辣的口感正正好男票比较能吃辣相对而言觉得他们家的麻辣口感麻有了辣还欠缺一点老板娘说考虑到客人口味不同所以没敢放太多辣椒能吃辣的朋友可以考虑下单之前和老板先说好鱼呢我们选的是黑鱼29斤的鱼加上一盆我以为没有什么东西实际上东西很多的锅底我们吃的饱饱的最后以为吃的差不多了打包一看简直像没动过一样分量还是满足的鱼比较新鲜伤心凉粉很辣不过口味也蛮好吃的总的来说性价比还是可以的两个人吃了大概160左右用了团购券的话一百块不到会考虑下次再来'

In [54]:
jieba.cut?